<a href="https://colab.research.google.com/github/gremlin97/EVA-8/blob/main/S5/Eva3_Step3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Target:**

Readded padding=1 back to convolutions to reduce the rapid decrease in feature maps resolution. Removed the extra output channels in the last layer and removed the maxpooling in the last block. Added 1 Convolution Layer to increase model capacity.

**Results:**

* Parameters: 7,386
* Best Train Accuracy: 99.07
* Best Test Accuracy: 99.20

**Analysis:**

Difference between training and test accuracy reduces substantially. Testing accuracy is more consistent in the last epochs. Training accuracy increases faster in a linear fashion along with test accuracy. Model params are reduced below 8k.

In [6]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [76]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1) # RF:1+(3-1)1=3; ji=1,jo=1; 28x28x1 -> 28x28x16
        self.bn1 = nn.BatchNorm2d(8)
        self.pool1 = nn.MaxPool2d(2, 2) # RF:3+(2-1)1=4; ji=1,jo=2; 28x28x16 -> 14x14x16
        self.drop1 = nn.Dropout(0.1)
        self.conv3 = nn.Conv2d(8, 16, 3, padding=1) # RF:4+(3-1)2=8; ji=2,jo=2; 14x14x16 -> 14x14x16
        self.bn2 = nn.BatchNorm2d(16)
        self.pool2 = nn.MaxPool2d(2, 2) # RF:8+(2-1)2=10; ji=2,jo=4; 14x14x16 -> 7x7x16
        self.drop2 = nn.Dropout(0.1)
        self.conv5 = nn.Conv2d(16, 16, 3) # RF:10+(3-1)4=18; ji=4,jo=4; 7x7x16 -> 5x5x16
        self.bn3 = nn.BatchNorm2d(16) 
        self.drop3 = nn.Dropout(0.1)
        self.conv6 = nn.Conv2d(16, 16, 3) # RF:18+(2-1)4=22; ji=4,jo=8; 5x5x16 -> 3x3x16
        self.bn4 = nn.BatchNorm2d(16)
        self.drop4 = nn.Dropout(0.1)
        self.conv7 = nn.Conv2d(16, 16, 3) # RF:18+(2-1)4=22; ji=4,jo=8; 5x5x16 -> 3x3x16
        self.gap = nn.AdaptiveAvgPool2d((1,1)) 

        self.lin = nn.Linear(16, 10)

    def forward(self, x):

        x = self.conv7(self.drop4(self.bn4(self.conv6(self.drop3(self.bn3(F.relu(self.conv5(self.drop2(self.pool2(self.bn2(F.relu(self.conv3(F.relu(self.drop1(self.pool1(self.bn1(F.relu(self.conv1(x)))))))))))))))))))

        x = self.gap(x)

        x = x.view(-1, 16)

        x = self.lin(x)
        
        #x = x.view(-1, 10)
        return F.log_softmax(x)

In [77]:
model = Net()
out = model(torch.randn(1,1,28,28))
print(out.shape)

torch.Size([1, 10])


<ipython-input-76-34307bc6583e>:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [78]:
model

Net(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop1): Dropout(p=0.1, inplace=False)
  (conv3): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop2): Dropout(p=0.1, inplace=False)
  (conv5): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
  (bn3): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop3): Dropout(p=0.1, inplace=False)
  (conv6): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1))
  (gap): AdaptiveAvgPool2d(output_size=(1, 1))
  (lin): Linear(in_features=16, out_features=10, bias=True)
)

In [79]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
         MaxPool2d-3            [-1, 8, 14, 14]               0
           Dropout-4            [-1, 8, 14, 14]               0
            Conv2d-5           [-1, 16, 14, 14]           1,168
       BatchNorm2d-6           [-1, 16, 14, 14]              32
         MaxPool2d-7             [-1, 16, 7, 7]               0
           Dropout-8             [-1, 16, 7, 7]               0
            Conv2d-9             [-1, 16, 5, 5]           2,320
      BatchNorm2d-10             [-1, 16, 5, 5]              32
          Dropout-11             [-1, 16, 5, 5]               0
           Conv2d-12             [-1, 16, 3, 3]           2,320
Adap

<ipython-input-76-34307bc6583e>:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [71]:
torch.manual_seed(1)
batch_size = 32

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.RandomRotation((-5.0, 5.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.RandomRotation((-5.0, 5.0), fill=(1,)),
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [72]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    correct = 0
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx} Train Accuracy={100. * correct / len(train_loader.dataset)}')

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [80]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.05, momentum=0.9)
scheduler =  optim.lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.1)

#lr=0.01

for epoch in range(15):
    print("Epoch: ",epoch+1)
    train(model, device, train_loader, optimizer, epoch)
    scheduler.step()
    test(model, device, test_loader)

Epoch:  1


  0%|          | 0/1875 [00:00<?, ?it/s]<ipython-input-76-34307bc6583e>:33: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.11352343112230301 batch_id=1874 Train Accuracy=94.14166666666667: 100%|██████████| 1875/1875 [00:30<00:00, 61.95it/s]



Test set: Average loss: 0.0667, Accuracy: 9778/10000 (98%)

Epoch:  2


loss=0.10847627371549606 batch_id=1874 Train Accuracy=96.96333333333334: 100%|██████████| 1875/1875 [00:29<00:00, 62.74it/s]



Test set: Average loss: 0.0458, Accuracy: 9857/10000 (99%)

Epoch:  3


...


Test set: Average loss: 0.0241, Accuracy: 9918/10000 (99%)

Epoch:  14


loss=0.02801971137523651 batch_id=1874 Train Accuracy=98.85833333333333: 100%|██████████| 1875/1875 [00:29<00:00, 63.89it/s]



Test set: Average loss: 0.0248, Accuracy: 9921/10000 (99%)

Epoch:  15


loss=0.16585306823253632 batch_id=1874 Train Accuracy=98.82333333333334: 100%|██████████| 1875/1875 [00:29<00:00, 63.20it/s]



Test set: Average loss: 0.0245, Accuracy: 9927/10000 (99%)

